# SCE-TTS: 음성합성 데모 (v1)

이 문서는 SCE-TTS 프로젝트의 음성 합성 데모입니다.

이 데모에 대한 더 자세한 정보는 아래 링크에서 확인하실 수 있습니다.  
https://sce-tts.github.io/#/v1/index

https://sce-tts.github.io/#/v1/test

## 1. 구글 드라이브 마운트

필요한 파일 5개 glow-tts의 config.json과 G_3511.pth이 필요합니다
mb-melgandml의 stats.npy파일과 config.yml이랑 
checkpoints파일안에 generator-667775.h5가 필요합니다

- `/content/drive/MyDrive/Watching_You_PJ/Project/06_Delieverable(산출물)/data/glow-tts/config.json`
- `/content/drive/MyDrive/Watching_You_PJ/Project/06_Delieverable(산출물)/data/glow-tts/G_3511.pth`
- `/content/drive/MyDrive/Watching_You_PJ/Project/06_Delieverable(산출물)/data/mb-melgan/stats.npy`
- `/content/drive/MyDrive/Watching_You_PJ/Project/06_Delieverable(산출물)/data/mb-melgan/config.yml`
- `/content/drive/MyDrive/Watching_You_PJ/Project/06_Delieverable(산출물)/data/mb-melgan/checkpoints/generator-667775.h5`




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. 필수 라이브러리 및 함수 불러오기

실행에 필요한 라이브러리 및 함수를 불러옵니다.

이 과정은 약 10분 정도 소요될 수 있습니다.

In [2]:
# Change CUDA version to 10.1
!rm /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

# check if installed successfully
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [3]:
import os
import sys
from pathlib import Path
from pprint import pprint

In [4]:
%tensorflow_version 2.x

In [5]:
%cd /content
!git clone --depth 1 https://github.com/sce-tts/g2pK.git
!git clone --depth 1 https://github.com/sce-tts/glow-tts.git
!git clone --depth 1 https://github.com/sce-tts/TensorflowTTS.git -b r0.7
!pip install -q --no-cache-dir "torch==1.5.1" -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install -q --no-cache-dir "cython==0.29.12" "tensorflow-gpu>=2.2.0" "tensorflow-addons>=0.9.1" "setuptools>=38.5.1" "librosa>=0.7.0" "soundfile>=0.10.2" "matplotlib>=3.1.0" "PyYAML>=3.12" "tqdm>=4.26.1" "h5py>=2.10.0" "pathos>=0.2.5" "unidecode>=1.1.1" "inflect>=4.1.0" "scikit-learn>=0.22.0" "pyworld>=0.2.10" "numba<=0.48" "numpy" "scipy" "pillow" "future" "konlpy" "jamo" "nltk" "python-mecab-ko"

/content
Cloning into 'g2pK'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 20 (delta 0), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.
Cloning into 'glow-tts'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 41 (delta 1), reused 38 (delta 1), pack-reused 0
Unpacking objects: 100% (41/41), done.
Cloning into 'TensorflowTTS'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 132 (delta 17), reused 99 (delta 3), pack-reused 0
Receiving objects: 100% (132/132), 7.60 MiB | 32.14 MiB/s, done.
Resolving deltas: 100% (17/17), done.
     |████████████████████████████████| 704.4 MB 1.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account a

In [6]:
%cd /content/glow-tts/monotonic_align
!python setup.py build_ext --inplace

/content/glow-tts/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/glow-tts/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'monotonic_align.core' extension
creating build
creating build/temp.linux-x86_64-3.7
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I/usr/include/python3.7m -c core.c -o build/temp.linux-x86_64-3.7/core.o
In file included from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/ndarraytypes

In [7]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # USE CPU

import yaml
import json
import numpy as np
import torch
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [8]:
sys.path.append('/content/TensorflowTTS')
from tensorflow_tts.processor.ljspeech import LJSpeechProcessor
from tensorflow_tts.processor.ljspeech import symbols as tensorflowtts_symbols
from tensorflow_tts.processor.ljspeech import _symbol_to_id

from tensorflow_tts.configs import MultiBandMelGANGeneratorConfig
from tensorflow_tts.models import TFMelGANGenerator
from tensorflow_tts.models import TFPQMF
sys.path.remove('/content/TensorflowTTS')

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [9]:
sys.path.append('/content/glow-tts')
from utils import HParams, load_checkpoint
from text import symbols as glow_tts_symbols, text_to_sequence
from audio_processing import dynamic_range_decompression
from models import FlowGenerator
sys.path.remove('/content/glow-tts')

In [10]:
SAMPLING_RATE = 22050
processor = LJSpeechProcessor(None, "korean_cleaners")

In [11]:
def load_glow_tts(config_path, checkpoint_path):
    with open(config_path, "r") as f:
        data = f.read()
    config = json.loads(data)

    hparams = HParams(**config)
    model = FlowGenerator(
        len(glow_tts_symbols),
        out_channels=hparams.data.n_mel_channels,
        **hparams.model
    ).to("cpu")

    load_checkpoint(checkpoint_path, model)
    model.decoder.store_inverse() # do not calcuate jacobians for fast decoding
    _ = model.eval()

    return model

def inference_glow_tts(text, model, noise_scale=0.333, length_scale=0.9):
    sequence = np.array(text_to_sequence(text, ['korean_cleaners']))[None, :]
    x_tst = torch.autograd.Variable(torch.from_numpy(sequence)).cpu().long()
    x_tst_lengths = torch.tensor([x_tst.shape[1]]).cpu()
    with torch.no_grad():
        (y_gen_tst, *r), attn_gen, *_ = model(x_tst, x_tst_lengths, gen=True, noise_scale=noise_scale, length_scale=length_scale)
    return y_gen_tst

In [12]:
def convert_mel(mel):
    converted = mel.float().data.cpu().numpy()
    converted = np.expand_dims(np.transpose(converted[0]), axis=0)
    return converted

def normalize_mel(mel, mean, sigma):
    normalized = dynamic_range_decompression(mel)
    normalized = convert_mel(normalized)
    normalized = (np.log10(normalized) - mean) / sigma
    return normalized

In [13]:
def load_mb_melgan(config_path, model_path):
    with open(config_path) as f:
        raw_config = yaml.load(f, Loader=yaml.Loader)
        mb_melgan_config = MultiBandMelGANGeneratorConfig(**raw_config["generator_params"])
        mb_melgan = TFMelGANGenerator(config=mb_melgan_config, name='melgan_generator')
        mb_melgan._build()
        mb_melgan.load_weights(model_path)
        pqmf = TFPQMF(config=mb_melgan_config, name="pqmf")
    return (mb_melgan, pqmf)

def load_stats(stats_path):
    mean, scale = np.load(stats_path)
    sigma = np.sqrt(scale)
    return mean, sigma

def synthesis(mb_melgan, pqmf, mel):
    generated_subbands = mb_melgan(mel)
    generated_audios = pqmf.synthesis(generated_subbands)
    return generated_audios[0, :, 0]

In [14]:
def generate_audio_glow_tts(text, noise_scale=0.333, length_scale=0.9):
    mel_original = inference_glow_tts(text, glow_tts, noise_scale, length_scale)
    mel_nomalized = normalize_mel(mel_original, mb_mean, mb_sigma)
    audio = synthesis(mb_melgan, pqmf, mel_nomalized)
    return audio

def generate_audio_fastspeech2(text):
    mel = inference_fastspeech2(text, fastspeech2)
    audio = synthesis(mb_melgan, pqmf, mel)
    return audio

In [15]:
import sys
import re
from unicodedata import normalize

sys.path.append('/content/g2pK')
import g2pk
sys.path.remove('/content/g2pK')

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [16]:
all_symbols = set(tensorflowtts_symbols + glow_tts_symbols)

g2p = g2pk.G2p()

In [17]:
def normalize_text(text):
    text = simple_replace(text)
    text = g2p.idioms(text)
    text = g2pk.english.convert_eng(text, g2p.cmu)
    text = g2pk.utils.annotate(text, g2p.mecab)
    text = g2pk.numerals.convert_num(text)
    text = re.sub("/[PJEB]", "", text)
    text = eng_cap(text)

    text = normalize('NFD', text)
    for pos, char in enumerate(text):
        if char not in all_symbols:
            text = text[:pos] + ' ' + text[pos + 1:]
    text = normalize('NFC', text)
    return text

def split_text(text):
    texts = []
    pi = 0
    for ci in range(len(text)):
        if text[ci] in ',;:':
            # replace character to dot
            text = text[:ci] + '.' + text[ci + 1:]
            continue
        if text[ci] in '.!?\n':
            texts.append(text[pi:ci + 1])
            pi = ci + 1
    texts.append(text[pi:])
    return texts

def eng_cap(text):
    text = re.sub(r'(a|A)', "에이", text)
    text = re.sub(r'(b|B)', "비", text)
    text = re.sub(r'(c|C)', "씨", text)
    text = re.sub(r'(d|D)', "디", text)
    text = re.sub(r'(e|E)', "이", text)
    text = re.sub(r'(f|F)', "에프", text)
    text = re.sub(r'(g|G)', "쥐", text)
    text = re.sub(r'(h|H)', "에이치", text)
    text = re.sub(r'(i|I)', "아이", text)
    text = re.sub(r'(j|J)', "제이", text)
    text = re.sub(r'(k|K)', "케이", text)
    text = re.sub(r'(l|L)', "엘", text)
    text = re.sub(r'(m|M)', "엠", text)
    text = re.sub(r'(n|N)', "엔", text)
    text = re.sub(r'(o|O)', "오", text)
    text = re.sub(r'(p|P)', "피", text)
    text = re.sub(r'(q|Q)', "큐", text)
    text = re.sub(r'(r|R)', "알", text)
    text = re.sub(r'(s|S)', "에스", text)
    text = re.sub(r'(t|T)', "티", text)
    text = re.sub(r'(u|U)', "유", text)
    text = re.sub(r'(v|V)', "브이", text)
    text = re.sub(r'(w|W)', "더블유", text)
    text = re.sub(r'(x|X)', "엑스", text)
    text = re.sub(r'(y|Y)', "와이", text)
    text = re.sub(r'(z|Z)', "지", text)

    return text

def simple_replace(text):
    # 중복된 문장 부호는 마지막 문장부호로 변경
    text = re.sub(r'[.?!]+\?', "?", text)
    text = re.sub(r'[.?!]+!', "!", text)
    text = re.sub(r'[.?!]+\.', ".", text)

    # 기본 자모음
    text = re.sub(r'ㄱ', "기역", text)
    text = re.sub(r'ㄴ', "니은", text)
    text = re.sub(r'ㄷ', "디귿", text)
    text = re.sub(r'ㄹ', "리을", text)
    text = re.sub(r'ㅁ', "미음", text)
    text = re.sub(r'ㅂ', "비읍", text)
    text = re.sub(r'ㅅ', "시옷", text)
    text = re.sub(r'ㅇ', "이응", text)
    text = re.sub(r'ㅈ', "지읒", text)
    text = re.sub(r'ㅊ', "치읓", text)
    text = re.sub(r'ㅋ', "키읔", text)
    text = re.sub(r'ㅌ', "티읕", text)
    text = re.sub(r'ㅍ', "피읖", text)
    text = re.sub(r'ㅎ', "히읗", text)
    text = re.sub(r'ㄲ', "쌍기역", text)
    text = re.sub(r'ㄸ', "쌍디귿", text)
    text = re.sub(r'ㅃ', "쌍비읍", text)
    text = re.sub(r'ㅆ', "쌍시옷", text)
    text = re.sub(r'ㅉ', "쌍지읒", text)
    text = re.sub(r'ㄳ', "기역시옷", text)
    text = re.sub(r'ㄵ', "니은지읒", text)
    text = re.sub(r'ㄶ', "니은히읗", text)
    text = re.sub(r'ㄺ', "리을기역", text)
    text = re.sub(r'ㄻ', "리을미음", text)
    text = re.sub(r'ㄼ', "리을비읍", text)
    text = re.sub(r'ㄽ', "리을시옷", text)
    text = re.sub(r'ㄾ', "리을티읕", text)
    text = re.sub(r'ㄿ', "리을피읍", text)
    text = re.sub(r'ㅀ', "리을히읗", text)
    text = re.sub(r'ㅄ', "비읍시옷", text)
    text = re.sub(r'ㅏ', "아", text)
    text = re.sub(r'ㅑ', "야", text)
    text = re.sub(r'ㅓ', "어", text)
    text = re.sub(r'ㅕ', "여", text)
    text = re.sub(r'ㅗ', "오", text)
    text = re.sub(r'ㅛ', "요", text)
    text = re.sub(r'ㅜ', "우", text)
    text = re.sub(r'ㅠ', "유", text)
    text = re.sub(r'ㅡ', "으", text)
    text = re.sub(r'ㅣ', "이", text)
    text = re.sub(r'ㅐ', "애", text)
    text = re.sub(r'ㅒ', "얘", text)
    text = re.sub(r'ㅔ', "에", text)
    text = re.sub(r'ㅖ', "예", text)
    text = re.sub(r'ㅘ', "와", text)
    text = re.sub(r'ㅙ', "왜", text)
    text = re.sub(r'ㅚ', "외", text)
    text = re.sub(r'ㅝ', "워", text)
    text = re.sub(r'ㅞ', "웨", text)
    text = re.sub(r'ㅟ', "위", text)
    text = re.sub(r'ㅢ', "의", text)

    return text

def process_text(text):
    texts = split_text(text)
    results = []
    for text in texts:
        text = normalize_text(text)
        if text:
            results.append(text)
    return results

## 3. 학습한 모델 불러오기

학습한 Glow-TTS와 Multi-band MelGAN 모델을 불러옵니다.

만약 다른 체크포인트에서 불러오시려면 아래 코드에서 경로를 적절하게 수정합니다.

In [19]:
glow_tts = load_glow_tts(
    config_path='/content/drive/MyDrive/Watching_You_PJ/Project/06_Delieverable(산출물)/data/glow-tts/config.json',
    checkpoint_path='/content/drive/MyDrive/Watching_You_PJ/Project/06_Delieverable(산출물)/data/glow-tts/G_3511.pth'
)

mb_mean, mb_sigma = load_stats(
    stats_path='/content/drive/MyDrive/Watching_You_PJ/Project/06_Delieverable(산출물)/data/mb-melgan/stats.npy'
)

mb_melgan, pqmf = load_mb_melgan(
    config_path='/content/drive/MyDrive/Watching_You_PJ/Project/06_Delieverable(산출물)/data/mb-melgan/config.yml',
    model_path='/content/drive/MyDrive/Watching_You_PJ/Project/06_Delieverable(산출물)/data/mb-melgan/checkpoints/generator-667775.h5'
)

INFO:root:Loaded checkpoint '/content/drive/MyDrive/Watching_You_PJ/Project/06_Delieverable(산출물)/data/glow-tts/G_3511.pth' (iteration 3511)
DEBUG:h5py._conv:Creating converter from 3 to 5


## 4. 음성 합성

실제 음성 합성을 수행합니다.

`long_text`의 값을 변경하여 다른 문장의 합성도 시도해보실 수 있습니다.

In [20]:
!pip install AudioSegment

     |████████████████████████████████| 66 kB 4.6 MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp37-cp37m-linux_x86_64.whl size=72369 sha256=74915431aaa12bb7f963dcc07f69ff65c1f22e77f595d7d619a8ae11d83dd73e
  Stored in directory: /root/.cache/pip/wheels/11/f9/67/a3158d131f57e1c0a7d8d966a707d4a2fb27567a4fe47723ad
Successfully built webrtcvad


In [21]:
from pydub import AudioSegment
long_text = """
청동기시대부터 사용된 청동기시대부터의 거울은 천상과 지상을 연결해주는 매개체였으며, 신라의 왕릉인 황남대총 남쪽무덤과 왕자의 고분인 금령총에서 각각 거울이 한 점씩 출토되었는데, 이는 신라의 거울이 천상과 지상을 연결해주는 매개체였기 때문이며, 이러한 거울은 천상과 지상을 연결해주는 매개체였으며, 이러한 거울의 성격은 삼국시대까지 그대로 이어졌다
"""
import IPython.display as ipd
for text in process_text(long_text):
    text = text.strip()
    if text:
        print(text)
        audio = generate_audio_glow_tts(text, noise_scale=0, length_scale=0.9)
        result = ipd.display(ipd.Audio(audio, rate=SAMPLING_RATE))
        print(type(result),result)
        

청동기시대부터 사용된 청동기시대부터의 거울은 천상과 지상을 연결해주는 매개체였으며. 신라의 왕릉인 황남대총 남쪽무덤과 왕자의 고분인 금령총에서 각각 거울이 한 점씩 출토되었는데. 이는 신라의 거울이 천상과 지상을 연결해주는 매개체였기 때문이며. 이러한 거울은 천상과 지상을 연결해주는 매개체였으며. 이러한 거울의 성격은 삼국시대까지 그대로 이어졌다


<class 'NoneType'> None


In [ ]:
with open('/content/test.wav', 'wb') as f:
    f.write(audio)

In [ ]:
result('/content/file_name_mp3/sound.wav')  # From file